In [1]:
from time import sleep

import torch

In [2]:
torch.cuda.is_available()

True

# Env

In [3]:
from gym_pusht.envs import PushTEnv

env = PushTEnv(render_mode="human")
observation, info = env.reset()

for _ in range(1000):
    action = env.action_space.sample()
    observation, reward, terminated, truncated, info = env.step(action)
    image = env.render()

    if terminated or truncated:
        observation, info = env.reset()

env.close()

2025-04-11 14:40:17.058 python[49066:674789] +[IMKClient subclass]: chose IMKClient_Modern
2025-04-11 14:40:17.058 python[49066:674789] +[IMKInputSession subclass]: chose IMKInputSession_Modern


240, 102

action = 154, 495

214 218